In [ ]:
# CELL 1: SETUP
import os, sys, shutil, subprocess
from google.colab import drive

print("🚀 Step 1: Installing Geospatial Tools...")

# 1. Install GDAL (System Tool)
if not shutil.which('gdalwarp'):
    subprocess.run(["sudo", "apt-get", "update"], stdout=subprocess.DEVNULL)
    subprocess.run(["sudo", "apt-get", "install", "-y", "gdal-bin"], stdout=subprocess.DEVNULL)

# 2. Install Rasterio (Python Library)
try:
    import rasterio
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rasterio"])
    import rasterio

# 3. Mount Drive
if not os.path.exists('/content/drive'):
    print("📂 Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("✓ Drive already mounted.")

print("✓ Setup Complete.")

🚀 Step 1: Installing Geospatial Tools...
✓ Drive already mounted.
✓ Setup Complete.


In [ ]:
# CELL 2: CONFIGURATION
# ==========================================
# INPUTS (Files on your Google Drive)
# ==========================================
# Folder containing the 87 DeepMind TIF files
DRIVE_TILES_FOLDER = '/content/drive/MyDrive/EarthEngine'

# Your Raw Wetland Labels File
DRIVE_RAW_LABELS = '/content/drive/MyDrive/bow_river_wetlands_10m_final.tif'

# ==========================================
# OUTPUTS (Where to save the result)
# ==========================================
# The final checkpoint file you need
OUTPUT_NPZ = '/content/drive/MyDrive/training_data_bow_river_FINAL.npz'

# ==========================================
# SETTINGS
# ==========================================
TARGET_SAMPLES = 100000   # How many pixels to collect
PIXELS_PER_BLOCK = 5000   # Higher number = Faster, but needs more RAM
BACKGROUND_VAL = 0        # The pixel value for "Not Wetland" in your labels

In [ ]:
# CELL 3: PREPARE & ALIGN
import os
import shutil
import glob
import rasterio

print("🚀 Step 3: Preparing Data...")

# 1. Create a temporary local folder (High speed)
LOCAL_DIR = '/content/fast_temp'
if os.path.exists(LOCAL_DIR): shutil.rmtree(LOCAL_DIR)
os.makedirs(LOCAL_DIR)

# 2. Build Virtual Mosaic (VRT)
# We map the Drive files to a local VRT file so we can treat them as one image
print("   Building Virtual Mosaic for 87 tiles...")
tif_files = glob.glob(os.path.join(DRIVE_TILES_FOLDER, '*.tif'))
if len(tif_files) == 0:
    raise Exception(f"❌ No TIF files found in {DRIVE_TILES_FOLDER}")

local_vrt_path = os.path.join(LOCAL_DIR, 'mosaic.vrt')
with open('tile_list.txt', 'w') as f:
    for t in tif_files: f.write(f"{t}\n")

# Execute GDAL Build VRT
os.system(f'gdalbuildvrt -input_file_list tile_list.txt "{local_vrt_path}"')

# 3. Copy Raw Labels to Local Disk (Fixes the GDAL 512 Error)
print("   Copying raw labels locally (for stability)...")
local_raw_labels = os.path.join(LOCAL_DIR, 'raw_labels.tif')
shutil.copy(DRIVE_RAW_LABELS, local_raw_labels)

# 4. Run Alignment (The Fix)
print("   Aligning labels to match satellite grid...")
local_aligned_labels = os.path.join(LOCAL_DIR, 'labels_aligned.tif')

# Get exact bounds from the VRT
with rasterio.open(local_vrt_path) as src:
    bounds = src.bounds
    width, height = src.width, src.height
    print(f"   Target Grid: {width}x{height}")

# Run gdalwarp using local paths
cmd = (f"gdalwarp -te {bounds.left} {bounds.bottom} {bounds.right} {bounds.top} "
       f"-ts {width} {height} -r near -dstnodata 0 -overwrite "
       f"'{local_raw_labels}' '{local_aligned_labels}'")

exit_code = os.system(cmd)

if exit_code == 0:
    print("✅ SUCCESS: Labels aligned successfully.")
else:
    raise Exception(f"❌ Alignment Failed with code {exit_code}")

🚀 Step 3: Preparing Data...
   Building Virtual Mosaic for 87 tiles...
   Copying raw labels locally (for stability)...
   Aligning labels to match satellite grid...
   Target Grid: 32694x22497
✅ SUCCESS: Labels aligned successfully.


In [ ]:
# CELL 4: DATA EXTRACTION
import numpy as np
print(f"🚀 Step 4: Extracting {TARGET_SAMPLES} samples...")
print("   (This reads from Drive, so it may take a few minutes...)")

X_chunks = []
y_chunks = []
samples_collected = 0

# Open the aligned files
with rasterio.open(local_vrt_path) as src_x, rasterio.open(local_aligned_labels) as src_y:
    # Get all blocks (windows)
    windows = [w for ij, w in src_y.block_windows(1)]
    np.random.shuffle(windows) # Shuffle to sample from everywhere

    for i, window in enumerate(windows):
        if samples_collected >= TARGET_SAMPLES: break

        # 1. Check Labels (Instant local read)
        y_data = src_y.read(1, window=window)
        valid_indices = np.where(y_data != BACKGROUND_VAL)

        # Skip empty blocks
        if len(valid_indices[0]) == 0: continue

        # 2. Select Random Pixels
        num_valid = len(valid_indices[0])
        sample_size = min(num_valid, PIXELS_PER_BLOCK)

        rand_idx = np.random.choice(num_valid, sample_size, replace=False)
        row_idx = valid_indices[0][rand_idx]
        col_idx = valid_indices[1][rand_idx]

        # 3. Fetch DeepMind Data (Slower Drive read)
        try:
            x_data = src_x.read(window=window) # (64, H, W)
            x_data_t = x_data.transpose(1, 2, 0) # (H, W, 64)

            X_sel = x_data_t[row_idx, col_idx, :]
            y_sel = y_data[row_idx, col_idx]

            # Filter NaNs
            valid_mask = ~np.isnan(X_sel).any(axis=1)

            if np.sum(valid_mask) > 0:
                X_chunks.append(X_sel[valid_mask])
                y_chunks.append(y_sel[valid_mask])
                samples_collected += np.sum(valid_mask)
                print(f"   ✓ Collected {samples_collected} / {TARGET_SAMPLES} samples...")

        except Exception as e:
            print(f"   ⚠ Error reading block: {e}")
            continue

# Combine and Save
if len(X_chunks) > 0:
    X = np.vstack(X_chunks)
    y = np.concatenate(y_chunks)

    # Trim to exact target
    if len(y) > TARGET_SAMPLES:
        idx = np.random.choice(len(y), TARGET_SAMPLES, replace=False)
        X = X[idx]
        y = y[idx]

    print(f"\n💾 Saving to {OUTPUT_NPZ}...")
    np.savez_compressed(OUTPUT_NPZ, X=X, y=y)
    print("✅ DONE! Checkpoint saved.")
else:
    print("❌ No valid data found. Check your label file.")

🚀 Step 4: Extracting 100000 samples...
   (This reads from Drive, so it may take a few minutes...)
   ✓ Collected 1288 / 100000 samples...
   ✓ Collected 1322 / 100000 samples...
   ✓ Collected 2880 / 100000 samples...
   ✓ Collected 3102 / 100000 samples...
   ✓ Collected 3987 / 100000 samples...
   ✓ Collected 5491 / 100000 samples...
   ✓ Collected 6395 / 100000 samples...
   ✓ Collected 6702 / 100000 samples...
   ✓ Collected 8570 / 100000 samples...
   ✓ Collected 8575 / 100000 samples...
   ✓ Collected 9800 / 100000 samples...
   ✓ Collected 11526 / 100000 samples...
   ✓ Collected 11903 / 100000 samples...
   ✓ Collected 12417 / 100000 samples...
   ✓ Collected 14797 / 100000 samples...
   ✓ Collected 15188 / 100000 samples...
   ✓ Collected 16679 / 100000 samples...
   ✓ Collected 16993 / 100000 samples...
   ✓ Collected 17730 / 100000 samples...
   ✓ Collected 18074 / 100000 samples...
   ✓ Collected 19634 / 100000 samples...
   ✓ Collected 21166 / 100000 samples...
   ✓ Colle

In [ ]:
# CELL 5: VERIFY
import numpy as np

if os.path.exists(OUTPUT_NPZ):
    data = np.load(OUTPUT_NPZ)
    print("🎉 File Loaded Successfully!")
    print(f"   X Shape: {data['X'].shape} (Features)")
    print(f"   y Shape: {data['y'].shape} (Labels)")
    print(f"   Saved at: {OUTPUT_NPZ}")
else:
    print("❌ File not found.")

🎉 File Loaded Successfully!
   X Shape: (100000, 64) (Features)
   y Shape: (100000,) (Labels)
   Saved at: /content/drive/MyDrive/training_data_bow_river_FINAL.npz
